# Social statistics

In [ ]:
import pandas as pd
X_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(X_train_path)


In [ ]:
# Extract the water withdrawal for 2021
X_train["date"] = pd.to_datetime(X_train["piezo_measurement_date"])
X_2021 = X_train[X_train["date"].dt.year == 2021]
prelev_df = []
for prelev in range(3):
    selected_cols = [f"prelev_longitude_{prelev}", f"prelev_latitude_{prelev}",f"prelev_volume_{prelev}", f"prelev_usage_label_{prelev}"]
    sub_prelev_df = X_2021[selected_cols]
    sub_prelev_df.columns = ["prelev_longitude","prelev_latitude","prelev_volume", "prelev_usage_label"]
    prelev_df.append(sub_prelev_df)

prelev_df = pd.concat(prelev_df, axis=0)

prelev_df.shape

In [ ]:
prelev_df = prelev_df.drop_duplicates()
prelev_df.shape

In [ ]:
# cartopy to display maps
%pip install cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [ ]:
#ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
def display_France_map(ax):
    """Display the map of France"""
    
    ax.set_extent([-5, 10, 42, 52], crs=ccrs.PlateCarree())
    # Draw the background
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.RIVERS)
    ax.gridlines(draw_labels=True)

In [ ]:
fig = plt.figure(figsize=(20, 15))
ax_1 = fig.add_subplot(121,  projection = ccrs.PlateCarree())

ax_2 = fig.add_subplot(122,  projection = ccrs.PlateCarree())

plt.show()